<a href="https://colab.research.google.com/github/InannaxX07/Mutual-Fund-tracker/blob/main/Mutual_fund_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
from datetime import datetime
import numpy as np
from google.colab import files

uploaded = files.upload()


Saving ZN250 - Monthly Portfolio November 2024.xlsx to ZN250 - Monthly Portfolio November 2024 (2).xlsx


In [10]:
import io
file_name = next(iter(uploaded))
print(f"Uploaded file name: {file_name}")

Uploaded file name: ZN250 - Monthly Portfolio November 2024 (1).xlsx


In [14]:
class MutualFundTracker:
    def __init__(self):

        self.holdings_data = {}
        self.funds_list = []

    def load_excel_data(self, fund_name: str, excel_data: pd.DataFrame, sheet_name: str = None) -> None:

        try:
            df = excel_data.copy()

            # to find the row index where actual data starts from
            start_idx = None
            header_keywords = ['Name of the Instrument', 'Instrument Name', 'Securities']

            for idx, row in df.iterrows():
                if any(isinstance(val, str) and any(keyword.lower() in val.lower()
                      for keyword in header_keywords) for val in row.values):
                    start_idx = idx
                    break

            if start_idx is None:
                raise ValueError("Could not find the start of data in the Excel file")


            df = df.iloc[start_idx:].reset_index(drop=True)

            # use first row as headers
            df.columns = df.iloc[0]
            df = df.iloc[1:].reset_index(drop=True)

            # remove fully empty rows
            df = df.dropna(how='all')

            print(f"\nData shape after cleaning: {df.shape}")
            print("\nColumns found:", df.columns.tolist())

            # map column names
            column_mappings = {
                'stock_name': ['Name of the Instrument', 'Security Name', 'Instrument Name', 'Securities'],
                'isin': ['ISIN', 'ISIN Code', 'Security Code'],
                'sector': ['Sector', 'Industry', 'Sector Classification'],
                'quantity': ['Quantity', 'No. of Shares', 'Holdings'],
                'value': ['Market Value', 'Value', 'Amount'],
                'weight': ['% to NAV', 'Weight', '% of NAV', 'Portfolio Weight']
            }

            # find the actual column names
            actual_columns = {}
            for target_col, possible_names in column_mappings.items():
                found = False
                for name in possible_names:
                    matching_cols = [col for col in df.columns if isinstance(col, str)
                                   and name.lower() in col.lower()]
                    if matching_cols:
                        actual_columns[target_col] = matching_cols[0]
                        found = True
                        break
                if not found:
                    raise ValueError(f"Could not find column matching {target_col}")

            print("\nMapped columns:", actual_columns)

            # convert data to std format
            holdings_df = pd.DataFrame({
                'date': datetime.now(),
                'stock_name': df[actual_columns['stock_name']],
                'isin': df[actual_columns['isin']],
                'sector': df[actual_columns['sector']],
                'quantity': pd.to_numeric(df[actual_columns['quantity']].astype(str).str.replace(',', ''),
                                       errors='coerce'),
                'value': pd.to_numeric(df[actual_columns['value']].astype(str).str.replace(',', ''),
                                     errors='coerce'),
                'weight': pd.to_numeric(df[actual_columns['weight']].astype(str).str.rstrip('%'),
                                      errors='coerce') / 100
            })

            # remove rows where ISIN is NaN
            holdings_df = holdings_df[holdings_df['isin'].notna()]

            # remove rows where stock_name contains 'Total' or similar keywords
            holdings_df = holdings_df[~holdings_df['stock_name'].str.contains(
                'Total|SUB TOTAL|GRAND TOTAL', na=False, case=False)]

            # store the data
            if fund_name not in self.funds_list:
                self.funds_list.append(fund_name)

            self.holdings_data[fund_name] = holdings_df
            print(f"\nSuccessfully loaded {len(holdings_df)} holdings for {fund_name}")

            # print sample
            print("\nSample of processed holdings:")
            print(holdings_df.head())

        except Exception as e:
            print(f"Error processing data: {str(e)}")
            raise

    def generate_report(self, fund_name: str) -> str:

        if fund_name not in self.holdings_data:
            return "No data available for the specified fund."

        current_data = self.holdings_data[fund_name]

        # basic portfolio stats
        stats = {
            'total_value': current_data['value'].sum(),
            'number_of_stocks': len(current_data),
            'number_of_sectors': len(current_data['sector'].unique()),
            'top_sectors': current_data.groupby('sector')['weight'].sum().nlargest(5),
            'top_holdings': current_data.nlargest(10, 'weight')
        }

        # report generation
        report = [f"Portfolio Analysis Report for {fund_name}"]
        report.append("=" * 50)

        # portfolio overview
        report.append("\nPortfolio Statistics:")
        report.append(f"Total Portfolio Value: ₹{stats['total_value']:,.2f} Lakhs")
        report.append(f"Number of Stocks: {stats['number_of_stocks']}")
        report.append(f"Number of Sectors: {stats['number_of_sectors']}")

        # top sectors
        report.append("\nTop 5 Sectors by Weight:")
        for sector, weight in stats['top_sectors'].items():
            if pd.notna(sector) and pd.notna(weight):
                report.append(f"  {sector}: {weight*100:.2f}%")

        # top holdings
        report.append("\nTop 10 Holdings:")
        for _, holding in stats['top_holdings'].iterrows():
            if pd.notna(holding['stock_name']) and pd.notna(holding['weight']):
                report.append(f"  {holding['stock_name']} ({holding['sector']}): {holding['weight']*100:.2f}%")

        return "\n".join(report)



In [15]:
tracker = MutualFundTracker()
try:
    tracker.load_excel_data("ZN250", df)
    report = tracker.generate_report("ZN250")
    print(report)
except Exception as e:
    print(f"Failed to process mutual fund data: {str(e)}")


Data shape after cleaning: (309, 9)

Columns found: [nan, nan, 'Name of the Instrument', 'ISIN', 'Rating / Industry^', 'Quantity', 'Market value\n(Rs. in Lakhs)', '% to NAV', 'YTM %']

Mapped columns: {'stock_name': 'Name of the Instrument', 'isin': 'ISIN', 'sector': 'Rating / Industry^', 'quantity': 'Quantity', 'value': 'Market value\n(Rs. in Lakhs)', 'weight': '% to NAV'}

Successfully loaded 253 holdings for ZN250

Sample of processed holdings:
                        date                   stock_name          isin  \
3 2025-01-22 11:58:02.508267            HDFC Bank Limited  INE040A01034   
4 2025-01-22 11:58:02.508267           ICICI Bank Limited  INE090A01021   
5 2025-01-22 11:58:02.508267  Reliance Industries Limited  INE002A01018   
6 2025-01-22 11:58:02.508267              Infosys Limited  INE009A01021   
7 2025-01-22 11:58:02.508267                  ITC Limited  INE154A01025   

               sector  quantity    value    weight  
3               Banks  199490.0  3582.94  0